In [36]:
import os
import numpy as np
import pickle
import subprocess as sp
import matplotlib.image as mpimg
import caffe 
import sys
import scipy


In [ ]:
caffe_root = '/home/ubuntu/caffe/'

sys.path.insert(0, caffe_root + 'python')

print 'setting up alexnet...'
caffe.set_mode_gpu()
alexnet = caffe.Classifier('/home/ubuntu/VideoNet/deploy.prototxt',
    caffe_root + 'models/bvlc_reference_caffenet/bvlc_reference_caffenet.caffemodel')

alexnet.transformer.set_mean('data', np.load(caffe_root + 'python/caffe/imagenet/ilsvrc_2012_mean.npy').mean(1).mean(1))
alexnet.transformer.set_raw_scale('data', 255)
alexnet.transformer.set_channel_swap('data', (2, 1, 0))

DATA_PATH = '/home/ubuntu/VideoNet/cnn_in_data_server_2seconds/' #path to the extracted frames (png) of each video of each class

class_names = sorted(os.listdir(DATA_PATH)) 

WRITE_PATH = '/home/ubuntu/VideoNet/cnn_out_data_fc6_2seconds_doubledata/' #path to write the output of CNN 10*4096 per video


print("generating CNN output...")

for class_index in range(len(class_names)):
    
    class_name = class_names[class_index]
    print('extracting for class %s' % class_name)

    write_dir = WRITE_PATH + class_name + '/'
    if not os.path.isdir(write_dir):
        os.mkdir(write_dir) 

    class_dir = DATA_PATH + class_name + '/'

    for pfile in sorted(os.listdir(class_dir)):

            batch = []
            for i in range(10): #we are using 10 frames per video
                if(i != 9):
                    filename = '0' + str(i+1) + '.png'
                elif(i == 9):
                    filename = str(i+1) + '.png'
                if not os.path.isfile(class_dir + pfile + '/' + filename):
                    filename = '01.png'
                    print 'Less than 10 images'
                img=mpimg.imread(class_dir + pfile + '/' + filename)
                temp = img.transpose(2,0,1)
                batch.append(temp)

            cnn_input = np.array(batch)
            
            #cnn_input.shape
            #without flipping
            cnn_out = alexnet.forward(**{alexnet.inputs[0]: cnn_input, 'blobs': ['fc6']})
            outlist = []
            for key, value in cnn_out.iteritems():
                temp_val = [value]
                if(key == 'fc6'):
                    outlist.append(temp_val)
            outlist = np.array(outlist)
            outlist = outlist.squeeze()
            write_name = pfile + '_cnn_output'    
            with open(write_dir + write_name, 'w') as w_pfile:
                    pickle.dump(outlist, w_pfile)

            #with flipping
            cnn_input = cnn_input.transpose(2,3,1,0)
            for i in range(cnn_input.shape[3]):
                cnn_input[:,:,:,i] = np.fliplr(cnn_input[:,:,:,i])
            cnn_input = cnn_input.transpose(3,2,0,1)
            
            cnn_out = alexnet.forward(**{alexnet.inputs[0]: cnn_input, 'blobs': ['fc6']})
            outlist = []
            for key, value in cnn_out.iteritems():
                temp_val = [value]
                if(key == 'fc6'):
                    outlist.append(temp_val)
            outlist = np.array(outlist)
            outlist = outlist.squeeze()
            write_name = pfile + '_cnn_output_flip'    
            with open(write_dir + write_name, 'w') as w_pfile:
                    pickle.dump(outlist, w_pfile)

setting up alexnet...
generating CNN output...
extracting for class ApplyEyeMakeup
extracting for class ApplyLipstick
extracting for class Archery
extracting for class BabyCrawling
extracting for class BalanceBeam